In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00a 0:00:01


In [3]:
import zipfile
import glob
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import random


from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch
from torchvision import datasets,models
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import v2
from torchvision.transforms import ColorJitter
from transformers import MaskFormerImageProcessor
import evaluate
from torch import nn
import torch.nn.functional as F
import wandb
import scipy


np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

2024-07-28 11:38:02.180837: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 11:38:02.180970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 11:38:02.319957: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
run_name = 'MaskFormer-cont2'
wandb.init(project='aimlproject', name=run_name,save_code=True, config={
            "epochs": 2,
           "batch_size": 12,
              "lr": 5e-5,
          "checkpoint": "facebook/maskformer-swin-tiny-ade",
           "model_name":"SegFormer", "early_stopping":False})  # mode='disabled',
config = wandb.config

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
id2label = {
            0: 'background',
            1: 'bird',
            2: 'boat',
            3: 'bottle',
            4: 'cat',
            5: 'chair',
            6: 'cow',
            7: 'dog',
            8: 'horse',
            9: 'person',
            10: 'pottedplant',
            11: 'sheep',
}

label2id = {v: k for k, v in id2label.items()}
model_checkpoint =  config.checkpoint


In [6]:
EXPR_PATH = '/kaggle/input/art-seg-ready/pascal_sbd_styled_expressionism2/kaggle/working/pascal_sbd_styled_expressionism'
IMPR_PATH = '/kaggle/input/art-seg-ready/pascal_sbd_styled_impressionism2/kaggle/working/pascal_sbd_styled_impressionism'
PIMP_PATH = '/kaggle/input/art-seg-ready/pascal_sbd_styled_post_impressionism2/kaggle/working/pascal_sbd_styled_post_impressionism'
REAL_PATH = '/kaggle/input/art-seg-ready/pascal_sbd_styled_realism2/kaggle/working/pascal_sbd_styled_realism'
PATHS = [ EXPR_PATH, IMPR_PATH, PIMP_PATH, REAL_PATH ]
PATHS_IM = [ os.path.join(path, 'images') for path in PATHS ]
PATHS_LAB = [ os.path.join(path, 'labels') for path in PATHS ]

In [7]:
PATHS

['/kaggle/input/art-seg-ready/pascal_sbd_styled_expressionism2/kaggle/working/pascal_sbd_styled_expressionism',
 '/kaggle/input/art-seg-ready/pascal_sbd_styled_impressionism2/kaggle/working/pascal_sbd_styled_impressionism',
 '/kaggle/input/art-seg-ready/pascal_sbd_styled_post_impressionism2/kaggle/working/pascal_sbd_styled_post_impressionism',
 '/kaggle/input/art-seg-ready/pascal_sbd_styled_realism2/kaggle/working/pascal_sbd_styled_realism']

In [ ]:
#print(os.listdir('/kaggle/input/art-seg-ready/pascal_sbd_styled_realism1/pascal_sbd_styled_realism1/kaggle/working/pascal_sbd_styled_realism/images'))
def plot_random_images(allstyles=True,path=''):
    selected_images = []
    if allstyles:
    #image_files = [f for f in os.listdir(basepath + '/dataset'+config.dataset+trainfolder) if f.startswith(startchar)]
        shuffled_paths = PATHS_IM
        random.shuffle(shuffled_paths)
        
        for i in range(len(PATHS)-1):
            path = shuffled_paths[i]
            image_files = os.listdir(path)
            image_files = random.sample(image_files, 2) 
            image_files = [ os.path.join(path, image) for image in image_files]
            selected_images += image_files
            
            
        path = shuffled_paths[3]

        image_files = os.listdir(path)
        image_files = random.sample(image_files,3)
        image_files = [ os.path.join(path, image) for image in image_files]
        selected_images += image_files
             
    else:
        if path == '':
            print('empty path, useless')
            return 
        image_files = os.listdir(path)
        image_files = random.sample(image_files, 9)
        image_files = [ os.path.join(path, image) for image in image_files]
        selected_images += image_files

    #selected_images = random.sample(image_files, 9)

    rows = 3
    cols = 3
    fig, axes = plt.subplots(rows, cols, figsize=(10, 10))

    # Plot each selected image
    for i, image_name in enumerate(selected_images):
        #img_path = os.path.join(basepath + '/dataset'+config.dataset+trainfolder, image_name)
        img = mpimg.imread(image_name)
        ax = axes[i // cols, i % cols]
        ax.imshow(img)
        ax.axis('off')

    # Adjust layout
    plt.tight_layout()
    plt.show()

plot_random_images()


In [9]:
train_list = []
for i in range(4):
    style_list = os.listdir(PATHS_LAB[i])
    style_list = [ os.path.join(PATHS_LAB[i], image) for image in style_list]
    train_list += style_list
    
    
#train_list,test_list = train_test_split(train_list, test_size=0.2, random_state=0)
train_list,val_list = train_test_split(train_list, test_size=0.2, random_state=0)

#print(train_list[1], train_list[-1])
example_conv = train_list[2020]
example_conv = example_conv[:-4] + '.jpg'
#example_conv = os.path.join()
example_conv = example_conv.replace("/labels/", "/images/")
print(example_conv)

/kaggle/input/art-seg-ready/pascal_sbd_styled_expressionism2/kaggle/working/pascal_sbd_styled_expressionism/images/2009_002240-stylized-expressionism_chaim-soutine_1937_9223372032559811545.jpg


In [10]:

class PaintingSegDataset(Dataset):
    def __init__(self, file_list, transform=None,train=True,use_gpu=torch.cuda.is_available()): #df(1st)
        self.file_list = file_list
        self.transform = transform
        self.filelength = len(file_list)
        self.jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 
        self.train = train
        self.use_gpu = use_gpu
        self.id_to_trainid = {
            0: 0,
            3: 1,
            4: 2,
            5: 3,
            8: 4,
            9: 5,
            10: 6,
            12: 7,
            13: 8,
            15: 9,
            16: 10,
            17: 11,
        }
        #if(train):
        #    self.folderpath = basepath +'/dataset' + config.dataset +trainfolder
        #else:
        #    self.folderpath = basepath +'/dataset'+config.dataset+testfolder


    def __len__(self):
        return self.filelength

    def __getitem__(self, idx):
        label_path = self.file_list[idx]
        
        img_path = label_path[:-4] + '.jpg'
        img_path = img_path.replace("/labels/", "/images/")
        
        img_orig = Image.open(img_path).convert('RGB')
        
        if label_path[-3:] == 'png':
            label = np.array(Image.open(label_path), dtype=np.float32)
        else:  # .mat
            label = scipy.io.loadmat(label_path)["GTcls"][0]['Segmentation'][0]
            
        
        
        label_copy = 255 * np.ones(label.shape, dtype=np.int8)
        for k, v in self.id_to_trainid.items():
            label_copy[label == k] = v
        
        label = Image.fromarray(label_copy)
        #label = label.convert('L')
        #print(label)
        img = self.jitter(img_orig)     
        
        #transform = v2.Compose([v2.Resize((512,512))])
        #img_orig = transform(img_orig)
        label_transform = v2.Compose([v2.Resize((512,512)),v2.PILToTensor()])
        label_tensor = label_transform(label).squeeze()
            
        return img, label,label_tensor #,img_orig

In [11]:
image_processor = MaskFormerImageProcessor.from_pretrained(model_checkpoint)

#test_dataset = PaintingSegDataset(test_list,transform=image_processor,train=False) #df
val_dataset = PaintingSegDataset(val_list,transform=image_processor,train=True) #df
train_dataset = PaintingSegDataset(train_list,transform=image_processor,train=True)

preprocessor_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/maskformer/image_processing_maskformer.py:412: FutureWarning: The `size_divisibility` argument is deprecated and will be removed in v4.27. Please use `size_divisor` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/maskformer/image_processing_maskformer.py:419: FutureWarning: The `max_size` argument is deprecated and will be removed in v4.27. Please use size['longest_edge'] instead.
  warnings.warn(


In [12]:
a,b,c = train_dataset[4]
print(a)

<PIL.Image.Image image mode=RGB size=504x376 at 0x7E421F701E70>


In [13]:
use_gpu = True

def collate_fn(batch):
    inputs = list(zip(*batch))
    img = inputs[0]
    labels = inputs[1]
    
    batch = image_processor(images=img, segmentation_maps=labels, return_tensors="pt",size=(512,512))
    
    return batch


def collate_fn_val(batch):
    inputs = list(zip(*batch))
    img = inputs[0]
    labels = inputs[1]
    
    batch = image_processor(images=img, segmentation_maps=labels, return_tensors="pt",size=(512,512))
    batch['gt_labels'] = inputs[2]
    
    return batch

batch_size = config.batch_size 
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn_val)
#test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

batch = next(iter(valid_loader))
#print(batch)

In [14]:
labels = [id2label[label] for label in batch["class_labels"][0].tolist()]



def visualize_mask(labels, label_name, sample_index):
    print("Label:", label_name)
    idx = labels.index(label_name)

    visual_mask = (batch["mask_labels"][sample_index][idx].bool().numpy() * 255).astype(np.uint8)
    image = batch["orig_img"][sample_index]
    #img = np.array(image.convert('L'))
    print(img.shape)
    #img = np.transpose(img)
    print(img.shape)
    #label = Image.fromarray(visual_mask)
    img = img * 0.5 + visual_mask * 0.5
    img = img.astype(np.uint8)

    plt.figure(figsize=(15, 10))
    plt.imshow(img)
    plt.show()
    return 

print(labels)
#visualize_mask(labels, "chair",0)

['background', 'chair']


In [15]:
from transformers import MaskFormerForInstanceSegmentation
import requests

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MaskFormerForInstanceSegmentation.from_pretrained(
    model_checkpoint,
    num_labels=12,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,  # Will ensure the segmentation specific components are reinitialized.
)
#model

model.load_state_dict(torch.load('/kaggle/input/maskformer-weights/pytorch/default/3/MaskFormer-cont10.pth'))


config.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Some weights of MaskFormerForInstanceSegmentation were not initialized from the model checkpoint at facebook/maskformer-swin-tiny-ade and are newly initialized because the shapes did not match:
- class_predictor.bias: found shape torch.Size([151]) in the checkpoint and torch.Size([13]) in the model instantiated
- class_predictor.weight: found shape torch.Size([151, 256]) in the checkpoint and torch.Size([13, 256]) in the model instantiated
- criterion.empty_weight: found shape torch.Size([151]) in the checkpoint and torch.Size([13]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [16]:
for i, (name, param) in enumerate(model.named_parameters()):
    if "encoder" in name:
        param.requires_grad = False
        
    #decoder_layers_to_freeze = ["0","1","2"]
    #if any(el in name for el in decoder_layers_to_freeze):
    #    param.requires_grad = False


for name, param in model.named_parameters():
    break
    #print(f"{name}: {param.requires_grad}")

In [17]:
from transformers import get_scheduler
from tqdm.auto import tqdm

optimizer = torch.optim.Adam(model.parameters(), lr=config.lr) #config.lr
model.to(device)
num_epochs = config.epochs
num_training_steps = num_epochs * len(train_loader)
num_val_steps = num_epochs * len(valid_loader)


lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [18]:
metric = evaluate.load("mean_iou")


progress_bar_train = tqdm(range(num_training_steps))
progress_bar_val = tqdm(range(num_val_steps))


model.train()
for epoch in range(num_epochs):
    training_loss_epoch = 0
    training_miou_epoch = 0
    
    for idx, batch in enumerate(train_loader):
        optimizer.zero_grad()

        #outputs = model(train_features_batch,train_labels_batch)
        outputs = model(
          pixel_values=batch["pixel_values"].to(device),
          mask_labels=[labels.to(device) for labels in batch["mask_labels"]],
          class_labels=[labels.to(device) for labels in batch["class_labels"]],
      )
        loss = outputs.loss # NllLoss2DBackward0
        
        training_loss_epoch += loss.item()
        #training_miou_epoch += metrics['mean_iou']
            
        loss.backward()
        optimizer.step() 
        lr_scheduler.step()
        progress_bar_train.update(1)
        
        
    validation_loss_epoch = 0
    validation_miou_epoch = 0
    
    for idx, batch in enumerate(valid_loader):

        with torch.no_grad():   
            pixel_values = batch["pixel_values"]
        
            outputs = model(
                pixel_values=pixel_values.to(device),
                mask_labels=[labels.to(device) for labels in batch["mask_labels"]],
                class_labels=[labels.to(device) for labels in batch["class_labels"]])

            validation_loss_epoch += loss.item()
            target_sizes = [(512,512) for image in pixel_values]
            predicted_segmentation_maps = image_processor.post_process_semantic_segmentation(outputs,
                                                                                  target_sizes=target_sizes)
            
            # get ground truth segmentation maps
            ground_truth_segmentation_maps = batch["gt_labels"]
            training_loss_epoch += loss.item()
            
            #print(ground_truth_segmentation_maps[0].shape)  
            #print(predicted_segmentation_maps[0].shape)

            metrics = metric.compute(references=ground_truth_segmentation_maps, predictions=predicted_segmentation_maps, num_labels=12, ignore_index=255)
            validation_miou_epoch += metrics['mean_iou']
            progress_bar_val.update(1)
            
            
    print("Saving model weights")
    torch.save(model.state_dict(), str(run_name) + str(epoch) + '.pth')
    
    print({'training_loss_epoch': training_loss_epoch/len(train_loader), 'validation_loss_epoch': validation_loss_epoch/len(valid_loader)})
    print({'validation_miou_epoch': validation_miou_epoch/len(valid_loader)})
    wandb.log({'training_loss_epoch': training_loss_epoch/len(train_loader), 'validation_loss_epoch': validation_loss_epoch/len(valid_loader),
               'validation_miou_epoch': validation_miou_epoch/len(valid_loader)})

  0%|          | 0/4460 [00:00<?, ?it/s]

  0%|          | 0/1116 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/datasets/features/image.py:348: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


Saving model weights
{'training_loss_epoch': 1.1755792117172292, 'validation_loss_epoch': 0.8417295217514038}
{'validation_miou_epoch': 0.24614224684767155}


/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


Saving model weights
{'training_loss_epoch': 1.195798421066438, 'validation_loss_epoch': 0.953302800655365}
{'validation_miou_epoch': 0.25223069744525256}


In [19]:
import gc
# VGGHUHIOdddggghhhhtttfffgjhghgjgjhgjfewgfw
model.cpu()
del model
gc.collect()
torch.cuda.empty_cache()

In [20]:
wandb.finish()

training_loss_epoch,▁█
validation_loss_epoch,▁█
validation_miou_epoch,▁█
training_loss_epoch,1.1958
validation_loss_epoch,0.9533
validation_miou_epoch,0.25223
